# Entropy in shallow (fixed depth) random 2D circuit

2D architecture with local 2-site gates applied for fixed time, reduced to a $m \times n$ ring of width $2d$

In [2]:
import numpy as np
import math
import tensornetwork as tn
from scipy.stats import unitary_group as ug
from ast import literal_eval

In [58]:
# TODO: consider simplified calculation based on Clifford gate set?

def random_2d_mps_bound(m,n,d,q=2):
    inner = [0,0,m,n]
    outer = [-1,-1,m+1,n+1]
    """
    Generates a MPS representation of the boundary of a m x n region of qudits with local dimension 2 after applying a local random circuit. Depth cannot exceed size of circuit.
    Returns dictionary of MPS nodes labeled by coordinate, qudit indices labeled by coordinate, list of edges for each of four cycles, list of layers of gates labeled by qudit, and dictionary of boundary condition nodes. 
    Coordinates for boundary of ring run from (0,0) to (m,n). MPS node axes 0-3 are Schmidt indices, and axis 4 is qudit indices.
    """
    # check if circuit is sufficiently shallow
    if (m < 4*d - 2 or n < 4*d - 2):
        print('Error: circuit depth is greater than size of area')
        return {}, {}, [], [], {}
    
    # initiate interior and exterior region
    mps = dict(zip([(i,0) for i in range(m)],[prod_state_node((1,1,1,1),q,name=str((i,0))) for i in range(m)]))
    mps.update(dict(zip([(m,i) for i in range(n)],[prod_state_node((1,1,1,1),q,name=str((m,i))) for i in range(n)])))
    mps.update(dict(zip([(m-i,n) for i in range(m)],[prod_state_node((1,1,1,1),q,name=str((m-i,n))) for i in range(m)])))
    mps.update(dict(zip([(0,n-i) for i in range(n)],[prod_state_node((1,1,1,1),q,name=str((0,n-i))) for i in range(n)])))
    mps.update(dict(zip([(i-1,-1) for i in range(m+2)],[prod_state_node((1,1,1,1),q,name=str((i-1,-1))) for i in range(m+2)])))
    mps.update(dict(zip([(m+1,i-1) for i in range(n+2)],[prod_state_node((1,1,1,1),q,name=str((m+1,i-1))) for i in range(n+2)])))
    mps.update(dict(zip([(m-i+1,n+1) for i in range(m+2)],[prod_state_node((1,1,1,1),q,name=str((m-i+1,n+1))) for i in range(m+2)])))
    mps.update(dict(zip([(-1,n-i+1) for i in range(n+2)],[prod_state_node((1,1,1,1),q,name=str((-1,n-i+1))) for i in range(n+2)])))
    qubits = {coord:mps[coord][-1] for coord in mps}
    edges = [[],[],[],[]]
    edges[1] += [mps[(i-1,0)][0]^mps[(i-1,-1)][2] for i in range(m+3)] #using list += doesn't allocate new object in memory but list = does
    edges[0] += [mps[(0,i-1)][3]^mps[(-1,i-1)][1] for i in range(n+3)]
    edges[2*((m+1)%2)] += [mps[(m+1,i-1)][3]^mps[(m,i-1)][1] for i in range(n+3)]
    edges[2*((n+1)%2)+1] += [mps[(i-1,n+1)][0]^mps[(i-1,n)][2] for i in range(m+3)]
    edges[0] += [mps[(2*i+2,-1)][3]^mps[(2*i+1,-1)][1] for i in range(m//2)]+[mps[(2*i+2,0)][3]^mps[(2*i+1,0)][1] for i in range(m//2)]
    edges[0] += [mps[(2*i+2,n)][3]^mps[(2*i+1,n)][1] for i in range(m//2)]+[mps[(2*i+2,n+1)][3]^mps[(2*i+1,n+1)][1] for i in range(m//2)]
    edges[2] += [mps[(2*i+1,-1)][3]^mps[(2*i,-1)][1] for i in range((m+1)//2)]+[mps[(2*i+1,0)][3]^mps[(2*i,0)][1] for i in range((m+1)//2)]
    edges[2] += [mps[(2*i+1,n)][3]^mps[(2*i,n)][1] for i in range((m+1)//2)]+[mps[(2*i+1,n+1)][3]^mps[(2*i,n+1)][1] for i in range((m+1)//2)]
    edges[1] += [mps[(-1,2*i+2)][0]^mps[(-1,2*i+1)][2] for i in range(n//2)]+[mps[(0,2*i+2)][0]^mps[(0,2*i+1)][2] for i in range(n//2)]
    edges[1] += [mps[(n,2*i+2)][0]^mps[(n,2*i+1)][2] for i in range(n//2)]+[mps[(n+1,2*i+2)][0]^mps[(n+1,2*i+1)][2] for i in range(n//2)]
    edges[3] += [mps[(-1,2*i+1)][0]^mps[(-1,2*i)][2] for i in range((n+1)//2)]+[mps[(0,2*i+1)][0]^mps[(0,2*i)][2] for i in range((n+1)//2)]
    edges[3] += [mps[(n,2*i+1)][0]^mps[(n,2*i)][2] for i in range((n+1)//2)]+[mps[(n+1,2*i+1)][0]^mps[(n+1,2*i)][2] for i in range((n+1)//2)]
    ops = []
        
    # generate random circuit
    for t in range(d):
        layer = []
        for j in [0,1,2,3]:
            c = {}
            # add necessary edges
            if (j == 0):
                if (t == 0) and (m%2):
                    mps.update(dict(zip([(m+2,i-1) for i in range(n+3)],[prod_state_node((1,1,1,1),q,name=str((m+2,i-1))) for i in range(n+3)])))
                    edges[0] += [mps[(m+2,i-1)][3]^mps[(m+1,i-1)][1] for i in range(n+3)]
                    edges[1] += [mps[(m+2,i+1)][0]^mps[(m+2,i)][2] for i in range(-1,n+1,2)]
                    edges[3] += [mps[(m+2,i+1)][0]^mps[(m+2,i)][2] for i in range(0,n+1,2)]
                    outer[2] += 1
                    mps.update(dict(zip([(m-1,i+1) for i in range(n-1)],[prod_state_node((1,1,1,1),q,name=str((m-1,i+1))) for i in range(n-1)])))
                    edges[0] += [mps[(m,i+1)][3]^mps[(m-1,i+1)][1] for i in range(n-1)]
                    edges[1] += [mps[(m-1,i+1)][0]^mps[(m-1,i)][2] for i in range(1,n,2)]
                    edges[3] += [mps[(m-1,i+1)][0]^mps[(m-1,i)][2] for i in range(0,n,2)]
                    inner[2] -= 1
                elif (t > 0):
                    h = outer[-1]-outer[1]
                    mps.update(dict(zip([(outer[0]-1,outer[1]+i) for i in range(h+1)],[prod_state_node((1,1,1,1),q,name=str((outer[0]-1,outer[1]+i))) for i in range(h+1)])))
                    mps.update(dict(zip([(outer[2]+1,outer[1]+i) for i in range(h+1)],[prod_state_node((1,1,1,1),q,name=str((outer[2]+1,outer[1]+i))) for i in range(h+1)])))
                    edges[0] += [mps[(outer[2]+1,outer[1]+i)][3]^mps[(outer[2],outer[1]+i)][1] for i in range(h+1)]
                    edges[0] += [mps[(outer[0],outer[1]+i)][3]^mps[(outer[0]-1,outer[1]+i)][1] for i in range(h+1)]
                    edges[1] += [mps[(outer[2]+1,i+1)][0]^mps[(outer[2]+1,i)][2] for i in range(2*((outer[0]+1)//2)-1,outer[-1],2)]
                    edges[1] += [mps[(outer[0]-1,i+1)][0]^mps[(outer[0]-1,i)][2] for i in range(2*((outer[0]+1)//2)-1,outer[-1],2)]
                    edges[3] += [mps[(outer[2]+1,i+1)][0]^mps[(outer[2]+1,i)][2] for i in range(2*((outer[0]+1)//2),outer[-1],2)]
                    edges[3] += [mps[(outer[0]-1,i+1)][0]^mps[(outer[0]-1,i)][2] for i in range(2*((outer[0]+1)//2),outer[-1],2)]
                    outer[0] -= 1
                    outer[2] += 1
                    h = inner[-1]-inner[1]
                    mps.update(dict(zip([(inner[0]+1,inner[1]+i+1) for i in range(h-1)],[prod_state_node((1,1,1,1),q,name=str((inner[0]+1,inner[1]+i+1))) for i in range(h-1)])))
                    mps.update(dict(zip([(inner[2]-1,inner[1]+i+1) for i in range(h-1)],[prod_state_node((1,1,1,1),q,name=str((inner[2]-1,inner[1]+i+1))) for i in range(h-1)])))
                    edges[0] += [mps[(inner[2],inner[1]+i+1)][3]^mps[(inner[2]-1,inner[1]+i+1)][1] for i in range(h-1)]
                    edges[0] += [mps[(inner[0]+1,inner[1]+i+1)][3]^mps[(inner[0],inner[1]+i+1)][1] for i in range(h-1)]
                    edges[1] += [mps[(inner[2]-1,i+1)][0]^mps[(inner[2]-1,i)][2] for i in range(inner[1],inner[-1],2)]
                    edges[1] += [mps[(inner[0]+1,i+1)][0]^mps[(inner[0]+1,i)][2] for i in range(inner[1],inner[-1],2)]
                    edges[3] += [mps[(inner[2]-1,i+1)][0]^mps[(inner[2]-1,i)][2] for i in range(inner[1]+1,inner[-1],2)]
                    edges[3] += [mps[(inner[0]+1,i+1)][0]^mps[(inner[0]+1,i)][2] for i in range(inner[1]+1,inner[-1],2)]
                    inner[0] += 1
                    inner[2] -= 1
            elif (j == 1):
                if (t == 0) and (n%2):
                    w = outer[2]-outer[0]
                    mps.update(dict(zip([(i-1,n+2) for i in range(w+1)],[prod_state_node((1,1,1,1),q,name=str((i-1,n+2))) for i in range(w+1)])))
                    edges[1] += [mps[(i-1,n+2)][0]^mps[(i-1,n+1)][2] for i in range(w+1)]
                    edges[0] += [mps[(i+1,n+2)][3]^mps[(i,n+2)][1] for i in range(outer[0],outer[2],2)]
                    edges[2] += [mps[(i+1,n+2)][3]^mps[(i,n+2)][1] for i in range(outer[0]+1,outer[2],2)]
                    outer[-1] += 1
                    w = inner[2]-inner[0]
                    mps.update(dict(zip([(i+1,n-1) for i in range(w-1)],[prod_state_node((1,1,1,1),q,name=str((i+1,n-1))) for i in range(w-1)])))
                    edges[1] += [mps[(i+1,n)][0]^mps[(i+1,n-1)][2] for i in range(w-1)]
                    edges[0] += [mps[(i+1,n-1)][3]^mps[(i,n-1)][1] for i in range(inner[0]+1,inner[2],2)]
                    edges[2] += [mps[(i+1,n-1)][3]^mps[(i,n-1)][1] for i in range(inner[0],inner[2],2)]
                    inner[-1] -= 1
                elif (t > 0):
                    w = outer[2]-outer[0]
                    mps.update(dict(zip([(i+outer[0],outer[-1]+1) for i in range(w+1)],[prod_state_node((1,1,1,1),q,name=str((i+outer[0],outer[-1]+1))) for i in range(w+1)])))
                    mps.update(dict(zip([(i+outer[0],outer[1]-1) for i in range(w+1)],[prod_state_node((1,1,1,1),q,name=str((i+outer[0],outer[1]-1))) for i in range(w+1)])))
                    edges[1] += [mps[(i+outer[0],outer[-1]+1)][0]^mps[(i+outer[0],outer[-1])][2] for i in range(w+1)]
                    edges[1] += [mps[(i+outer[0],outer[1])][0]^mps[(i+outer[0],outer[1]-1)][2] for i in range(w+1)]
                    edges[0] += [mps[(i+1,outer[-1]+1)][3]^mps[(i,outer[-1]+1)][1] for i in range(outer[0],outer[2],2)]
                    edges[0] += [mps[(i+1,outer[1]-1)][3]^mps[(i,outer[1]-1)][1] for i in range(outer[0],outer[2],2)]
                    edges[2] += [mps[(i+1,outer[-1]+1)][3]^mps[(i,outer[-1]+1)][1] for i in range(outer[0]+1,outer[2],2)]
                    edges[2] += [mps[(i+1,outer[1]-1)][3]^mps[(i,outer[1]-1)][1] for i in range(outer[0]+1,outer[2],2)]
                    outer[-1] += 1
                    outer[1] -= 1
                    w = inner[2]-inner[0]
                    mps.update(dict(zip([(inner[0]+i+1,inner[1]+1) for i in range(w-1)],[prod_state_node((1,1,1,1),q,name=str((inner[0]+i+1,inner[1]+1))) for i in range(w-1)])))
                    mps.update(dict(zip([(inner[0]+i+1,inner[-1]-1) for i in range(w-1)],[prod_state_node((1,1,1,1),q,name=str((inner[0]+i+1,inner[-1]-1) for i in range(w-1))) for i in range(w-1)])))
                    edges[1] += [mps[(i+1+inner[0],inner[-1])][0]^mps[(i+1+inner[0],inner[-1]-1)][2] for i in range(w-1)]
                    edges[1] += [mps[(i+1+inner[0],inner[1]-1)][0]^mps[(i+1+inner[0],inner[1])][2] for i in range(w-1)]
                    edges[0] += [mps[(i+1,inner[-1]-1)][3]^mps[(i,inner[-1]-1)][1] for i in range(inner[0]+1,inner[2],2)]
                    edges[0] += [mps[(i+1,inner[1]+1)][3]^mps[(i,inner[1]+1)][1] for i in range(inner[0]+1,inner[2],2)]
                    edges[2] += [mps[(i+1,inner[-1]-1)][3]^mps[(i,inner[-1]-1)][1] for i in range(inner[0],inner[2],2)]
                    edges[2] += [mps[(i+1,inner[1]+1)][3]^mps[(i,inner[1]+1)][1] for i in range(inner[0],inner[2],2)]
                    inner[-1] -= 1
                    inner[1] += 1
            elif (j == 2):
                h = outer[-1]-outer[1]
                mps.update(dict(zip([(outer[0]-1,outer[1]+i) for i in range(h+1)],[prod_state_node((1,1,1,1),q,name=str((outer[0]-1,outer[1]+i))) for i in range(h+1)])))
                mps.update(dict(zip([(outer[2]+1,outer[1]+i) for i in range(h+1)],[prod_state_node((1,1,1,1),q,name=str((outer[2]+1,outer[1]+i))) for i in range(h+1)])))
                edges[2] += [mps[(outer[2]+1,outer[1]+i)][3]^mps[(outer[2],outer[1]+i)][1] for i in range(h+1)]
                edges[2] += [mps[(outer[0],outer[1]+i)][3]^mps[(outer[0]-1,outer[1]+i)][1] for i in range(h+1)]
                edges[3] += [mps[(outer[2]+1,i+1)][0]^mps[(outer[2]+1,i)][2] for i in range(2*((outer[0]+1)//2)-1,outer[-1],2)]
                edges[3] += [mps[(outer[0]-1,i+1)][0]^mps[(outer[0]-1,i)][2] for i in range(2*((outer[0]+1)//2)-1,outer[-1],2)]
                edges[1] += [mps[(outer[2]+1,i+1)][0]^mps[(outer[2]+1,i)][2] for i in range(2*((outer[0]+1)//2),outer[-1],2)]
                edges[1] += [mps[(outer[0]-1,i+1)][0]^mps[(outer[0]-1,i)][2] for i in range(2*((outer[0]+1)//2),outer[-1],2)]
                outer[0] -= 1
                outer[2] += 1
                h = inner[-1]-inner[1]
                mps.update(dict(zip([(inner[0]+1,inner[1]+i+1) for i in range(h-1)],[prod_state_node((1,1,1,1),q,name=str((inner[0]+1,inner[1]+i+1))) for i in range(h-1)])))
                mps.update(dict(zip([(inner[2]-1,inner[1]+i+1) for i in range(h-1)],[prod_state_node((1,1,1,1),q,name=str((inner[2]-1,inner[1]+i+1))) for i in range(h-1)])))
                edges[2] += [mps[(inner[2],inner[1]+i+1)][3]^mps[(inner[2]-1,inner[1]+i+1)][1] for i in range(h-1)]
                edges[2] += [mps[(inner[0]+1,inner[1]+i+1)][3]^mps[(inner[0],inner[1]+i+1)][1] for i in range(h-1)]
                edges[3] += [mps[(inner[2]-1,i+1)][0]^mps[(inner[2]-1,i)][2] for i in range(inner[1],inner[-1],2)]
                edges[3] += [mps[(inner[0]+1,i+1)][0]^mps[(inner[0]+1,i)][2] for i in range(inner[1],inner[-1],2)]
                edges[1] += [mps[(inner[2]-1,i+1)][0]^mps[(inner[2]-1,i)][2] for i in range(inner[1]+1,inner[-1],2)]
                edges[1] += [mps[(inner[0]+1,i+1)][0]^mps[(inner[0]+1,i)][2] for i in range(inner[1]+1,inner[-1],2)]
                inner[0] += 1
                inner[2] -= 1
            else:
                w = outer[2]-outer[0]
                mps.update(dict(zip([(i+outer[0],outer[-1]+1) for i in range(w+1)],[prod_state_node((1,1,1,1),q,name=str((i+outer[0],outer[-1]+1))) for i in range(w+1)])))
                mps.update(dict(zip([(i+outer[0],outer[1]-1) for i in range(w+1)],[prod_state_node((1,1,1,1),q,name=str((i+outer[0],outer[1]-1))) for i in range(w+1)])))
                edges[3] += [mps[(i+outer[0],outer[-1]+1)][0]^mps[(i+outer[0],outer[-1])][2] for i in range(w+1)]
                edges[3] += [mps[(i+outer[0],outer[1])][0]^mps[(i+outer[0],outer[1]-1)][2] for i in range(w+1)]
                edges[2] += [mps[(i+1,outer[-1]+1)][3]^mps[(i,outer[-1]+1)][1] for i in range(outer[0],outer[2],2)]
                edges[2] += [mps[(i+1,outer[1]-1)][3]^mps[(i,outer[1]-1)][1] for i in range(outer[0],outer[2],2)]
                edges[0] += [mps[(i+1,outer[-1]+1)][3]^mps[(i,outer[-1]+1)][1] for i in range(outer[0]+1,outer[2],2)]
                edges[0] += [mps[(i+1,outer[1]-1)][3]^mps[(i,outer[1]-1)][1] for i in range(outer[0]+1,outer[2],2)]
                outer[-1] += 1
                outer[1] -= 1
                w = inner[2]-inner[0]
                mps.update(dict(zip([(inner[0]+i+1,inner[1]+1) for i in range(w-1)],[prod_state_node((1,1,1,1),q,name=str((inner[0]+i+1,inner[1]+1))) for i in range(w-1)])))
                mps.update(dict(zip([(inner[0]+i+1,inner[-1]-1) for i in range(w-1)],[prod_state_node((1,1,1,1),q,name=str((inner[0]+i+1,inner[-1]-1) for i in range(w-1))) for i in range(w-1)])))
                edges[3] += [mps[(i+1+inner[0],inner[-1])][0]^mps[(i+1+inner[0],inner[-1]-1)][2] for i in range(w-1)]
                edges[3] += [mps[(i+1+inner[0],inner[1]-1)][0]^mps[(i+1+inner[0],inner[1])][2] for i in range(w-1)]
                edges[2] += [mps[(i+1,inner[-1]-1)][3]^mps[(i,inner[-1]-1)][1] for i in range(inner[0]+1,inner[2],2)]
                edges[2] += [mps[(i+1,inner[1]+1)][3]^mps[(i,inner[1]+1)][1] for i in range(inner[0]+1,inner[2],2)]
                edges[0] += [mps[(i+1,inner[-1]-1)][3]^mps[(i,inner[-1]-1)][1] for i in range(inner[0],inner[2],2)]
                edges[0] += [mps[(i+1,inner[1]+1)][3]^mps[(i,inner[1]+1)][1] for i in range(inner[0],inner[2],2)]
                inner[-1] -= 1
                inner[1] += 1
            
            # add random gates
            for e in edges[j]:
                a,b = e.get_nodes()
                Ue = random_2_site(q,name=a.name+','+b.name)
                qubits[literal_eval(a.name)] ^ Ue[0]
                qubits[literal_eval(b.name)] ^ Ue[1]
                qubits[literal_eval(a.name)] = Ue[2]
                qubits[literal_eval(b.name)] = Ue[3]
                c[a.name+','+b.name]=Ue
            layer.append(c)
        ops.append(layer)
    
    # contract loose edges with boundary conditions
    wo = outer[2]-outer[0]
    wi = inner[2]-inner[0]
    ho = outer[-1]-outer[1]
    hi = inner[-1]-inner[1]
    bound = dict(zip([(outer[0]+i,outer[1]) for i in range(wo+1)],[tn.Node(np.asarray([1])) for i in range(wo+1)]))
    bound.update(dict(zip([(outer[0]+i,outer[-1]) for i in range(wo+1)],[tn.Node(np.asarray([1])) for i in range(wo+1)])))
    bound.update(dict(zip([(outer[2],outer[1]+i,1) for i in range(ho+1)],[tn.Node(np.asarray([1])) for i in range(ho+1)])))
    bound.update(dict(zip([(outer[0],outer[1]+i,1) for i in range(ho+1)],[tn.Node(np.asarray([1])) for i in range(ho+1)])))
    bound.update(dict(zip([(inner[0]+i+1,inner[1]) for i in range(wi-1)],[tn.Node(np.asarray([1])) for i in range(wi-1)])))
    bound.update(dict(zip([(inner[0]+i+1,inner[-1]) for i in range(wi-1)],[tn.Node(np.asarray([1])) for i in range(wi-1)])))
    bound.update(dict(zip([(inner[0],inner[1]+i+1) for i in range(hi-1)],[tn.Node(np.asarray([1])) for i in range(hi-1)])))
    bound.update(dict(zip([(inner[2],inner[1]+i+1) for i in range(hi-1)],[tn.Node(np.asarray([1])) for i in range(hi-1)])))
    for i in range(wo+1):
        bound[(outer[0]+i,outer[1])][0] ^ mps[(outer[0]+i,outer[1])][0]
        bound[(outer[0]+i,outer[-1])][0] ^ mps[(outer[0]+i,outer[-1])][2]
    for i in range(wi-1):
        bound[(inner[0]+i+1,inner[1])][0] ^ mps[(inner[0]+i+1,inner[1])][2]
        bound[(inner[0]+i+1,inner[-1])][0] ^ mps[(inner[0]+i+1,inner[-1])][0]
    for i in range(ho+1):
#         print(outer[(outer[2],outer[1]+i)].get_all_edges())
        bound[(outer[2],outer[1]+i,1)][0] ^ mps[(outer[2],outer[1]+i)][1]
        bound[(outer[0],outer[1]+i,1)][0] ^ mps[(outer[0],outer[1]+i)][3]
    for i in range(hi-1):
        bound[(inner[0],inner[1]+i+1)][0] ^ mps[(inner[0],inner[1]+i+1)][1]
        bound[(inner[2],inner[1]+i+1)][0] ^ mps[(inner[2],inner[1]+i+1)][3]
    return mps, qubits, edges, ops, bound

def prod_state_node(ranks,q=2,name=None):
    """
    Returns a MPS representation of a node in a product state with local dimension q, where ranks is a listlike that defines Schmidt indices.
    """
    return tn.Node(np.asarray([np.ones(shape=ranks)]+[np.zeros(shape=ranks) for _ in range(q-1)]).swapaxes(0,-1),name=name)

def random_2_site(q=2,name=None):
    """
    Returns a Haar-random 2-site gate as a node with axes 0-1 as input and axes 2-3 as output.
    """
    U = ug.rvs(q**2)
    U.resize((q,q,q,q))
    return tn.Node(U,name=name)

In [64]:
mps, qubits, edges, ops, bound = random_2d_mps_bound(9,9,0,q=2)
len([c for c in qubits])

80

In [51]:
# 1D product state
a = [tn.Node(np.asarray([np.ones(shape=(1,1)),np.zeros(shape=(1,1))]).swapaxes(0,-1)) for j in range(3)]
e = [a[j][1] ^ a[j+1][0] for j in range(2)]
i = tn.Node(np.asarray([1]))
o = tn.Node(np.asarray([1]))
e.append(a[0][0] ^ i[0])
e.append(a[-1][1] ^ o[0])
io = [tn.Node(np.asarray([1,0])) for j in range(3)]
for j in range(3):
    e.append(io[j][0] ^ a[j][-1])
result = a[0] @ i
# print(result.get_tensor())
for j in range(2):
    result = result @ a[j+1]
#     print(result.get_tensor())
    result = result @ io[j]
#     print(result.get_tensor())
result = result @ o @ io[-1]
print(result.get_tensor())

1.0


In [8]:
# to demonstrate computation with random 2-site gate
U = ug.rvs(2**2)
V = np.copy(U)
# Ud = np.conjugate(U.T)
U.resize((2,2,2,2))
# Ud.resize((2,2,2,2))
Ud = np.conjugate(U)
Un = tn.Node(U)
Udn = tn.Node(Ud)
a = [tn.Node(np.asarray([np.ones(shape=(1,1)),np.zeros(shape=(1,1))]).swapaxes(0,-1)) for j in range(2)]
b = [tn.Node(np.asarray([np.ones(shape=(1,1)),np.zeros(shape=(1,1))]).swapaxes(0,-1)) for j in range(2)]
a[0][1] ^ a[1][0]
b[0][1] ^ b[1][0]
a[0][-1] ^ Un[0]
a[1][-1] ^ Un[1]
Un[2] ^ Udn[2]
Un[3] ^ Udn[3]
b[0][-1] ^ Udn[0]
b[1][-1] ^ Udn[1]

ia = tn.Node(np.asarray([1]))
oa = tn.Node(np.asarray([1]))
ib = tn.Node(np.asarray([1]))
ob = tn.Node(np.asarray([1]))
a[0][0] ^ ia[0]
a[-1][1] ^ oa[0]
b[0][0] ^ ib[0]
b[-1][1] ^ ob[0]

result = ia @ a[0] @ a[1] @ oa
asdf = np.copy(result.get_tensor())
asdf.resize((4,1))
print(asdf)
print()
print(V)
print()
print(V * asdf)
print()
result = result @ Un 
print(result.get_tensor())
print()
result = result @ Udn @ b[0] @ b[1] @ ib @ ob
print(result.get_tensor())

[[1.]
 [0.]
 [0.]
 [0.]]

[[ 0.61491228-0.35087006j -0.07537495-0.32771485j  0.35153713-0.26769494j
   0.27725511-0.33702427j]
 [ 0.26723154-0.13752608j -0.23933267+0.20032495j  0.49931714+0.10362269j
  -0.60704699+0.42860513j]
 [ 0.59174243-0.00149927j  0.00606351+0.61125233j -0.52315095-0.0283031j
  -0.00383622-0.04086281j]
 [-0.1728335 -0.1685651j  -0.50633974+0.39933104j  0.24644415+0.45769324j
   0.4919759 -0.11666648j]]

[[ 0.61491228-0.35087006j -0.07537495-0.32771485j  0.35153713-0.26769494j
   0.27725511-0.33702427j]
 [ 0.        +0.j         -0.        +0.j          0.        +0.j
  -0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        -0.j
   0.        -0.j        ]
 [ 0.        -0.j         -0.        +0.j          0.        +0.j
   0.        +0.j        ]]

[[ 0.61491228-0.35087006j  0.35153713-0.26769494j]
 [-0.07537495-0.32771485j  0.27725511-0.33702427j]]

(0.9999999999999992+1.7822917817126497e-17j)
